In [3]:
import os
import requests
import time
from datetime import datetime, timedelta

import yaml

import re
import pandas as pd

# Download zip files from URLs

## Variables

In [4]:
# Open the YAML file and load its contents into a dictionary
with open('../../references/config_notebook.yaml', 'r') as f:
    config = yaml.safe_load(f)

# Access the variables in the dictionary
my_vars = config

In [5]:
duration = my_vars['LOCAL']['duration']
market_data = my_vars['LOCAL']['market_data']
symbol = my_vars['LOCAL']['symbol']
folder_path = my_vars['SOURCES']['binance']['BTCUSDT']
#colnames = my_vars['LOCAL']['colnames']
saving_path = my_vars['SOURCES']['binance']['BTCUSDT']

In [13]:
start_date = re.search("([0-9]{4}\-[0-9]{2}\-[0-9]{2})",sorted(os.listdir(folder_path), reverse=True)[0]).group(1)
today_date = '2023-03-24'
api_url = f"{my_vars['SOURCES']['binance']['url']}/{duration}/{market_data}/{symbol}/"
url = api_url + f'{symbol}-{market_data}-{start_date}.zip'
output = f'{saving_path}/{symbol}-{market_data}-{start_date}.zip'

# Checking file downloaded 

# Fetch the full historical record 

## Dealing with missing dates

In [6]:
hist = os.listdir(f'{saving_path}')

In [7]:
hist_date = [re.search("([0-9]{4}\-[0-9]{2}\-[0-9]{2})", x).group(1) for x in hist if re.search("([0-9]{4}\-[0-9]{2}\-[0-9]{2})", x) is not None]
hist_date.sort() 

In [8]:
hist_set = [datetime.strptime(x,'%Y-%m-%d') for x in hist_date]

In [9]:
date_set = set(hist_set[0] + timedelta(x) for x in range(( datetime.today() - hist_set[0]).days))

In [15]:
missing = sorted(date_set - set(hist_set))
missing

[datetime.datetime(2023, 6, 17, 0, 0),
 datetime.datetime(2023, 6, 18, 0, 0),
 datetime.datetime(2023, 6, 19, 0, 0),
 datetime.datetime(2023, 6, 20, 0, 0),
 datetime.datetime(2023, 6, 21, 0, 0),
 datetime.datetime(2023, 6, 22, 0, 0),
 datetime.datetime(2023, 6, 23, 0, 0),
 datetime.datetime(2023, 6, 24, 0, 0),
 datetime.datetime(2023, 6, 25, 0, 0),
 datetime.datetime(2023, 6, 26, 0, 0),
 datetime.datetime(2023, 6, 27, 0, 0),
 datetime.datetime(2023, 6, 28, 0, 0),
 datetime.datetime(2023, 6, 29, 0, 0),
 datetime.datetime(2023, 6, 30, 0, 0),
 datetime.datetime(2023, 7, 1, 0, 0),
 datetime.datetime(2023, 7, 2, 0, 0),
 datetime.datetime(2023, 7, 3, 0, 0),
 datetime.datetime(2023, 7, 4, 0, 0),
 datetime.datetime(2023, 7, 5, 0, 0),
 datetime.datetime(2023, 7, 6, 0, 0),
 datetime.datetime(2023, 7, 7, 0, 0),
 datetime.datetime(2023, 7, 8, 0, 0),
 datetime.datetime(2023, 7, 9, 0, 0),
 datetime.datetime(2023, 7, 10, 0, 0),
 datetime.datetime(2023, 7, 11, 0, 0),
 datetime.datetime(2023, 7, 12, 0,

## Fetching data

In [14]:
for x in missing:
    url = api_url + f'{symbol}-{market_data}-{x.date()}.zip'
    output = f'{saving_path}/{symbol}-{market_data}-{x.date()}.zip'
    try : 
        res = requests.get(url)
        res.raise_for_status() 
        with open(output, 'wb') as f:
            f.write(res.content)
        time.sleep(5)
    except requests.exceptions.HTTPError as e:
        print(f'HTTP error occurred: {e}')
    except requests.exceptions.RequestException as e:
        print(f'An error occurred: {e}')

# Read

**Trade**
- Id: trade ID
- price: price
- qty: quantity
- quoteQty: quote quantity
- time: time in unix time format
- isBuyerMaker:	was the buyer the maker
- isBestMatch:	was the trade the best price match


    isBuyerMaker = true -> SELL
    isBuyerMaker = false -> BUY

Market Orders using quantity, is when you want to buy or sell a specific amount of the base asset.

Market Orders using quoteOrderQty is when you want to spend or receive a specific amount of the quote asse

In [ ]:
colnames = ['Id','price','qty','quoteQty','time','isBuyerMaker','isBestMatch']
df = pd.read_csv(f'{output}',compression='zip', names=colnames, header=None)
df.index = pd.to_datetime(df.index, unit='ms')
df.head()

In [ ]:
df.tail()

In [ ]:
df.describe()